In [0]:
%sql
create catalog if not exists dev
managed location 'abfss://unity-catalog-storage@dbstorage6zhjt2acugja6.dfs.core.windows.net/3140462481804720'

In [0]:
%sql
create database if not exists dev.demodb


In [0]:
%sql
drop database if exists dev.default cascade

In [0]:
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

flight_time_dt = spark.read.format("json").schema(flight_schema_ddl).\
        option("dateFormat", "M/d/y").\
        load('abfss://dbstorage@skstorage15062025.dfs.core.windows.net/external/flight-time.json')


In [0]:
display(flight_time_dt)